In [1]:
import torch
import numpy as np
from model import UNet
from dataset import CarvanaDataset
from torchvision import transforms
from matplotlib import pyplot as plt
from torch.utils.data import DataLoader
import transforms as tsfms
import segmentation_models_pytorch as smp
import metrics

In [2]:
checkpoint = torch.load('checkpoint_overfit.pth')

In [ ]:
# model_ = UNet(3, 2)
# model_.load_state_dict(checkpoint['model_state_dict'])
model1 = smp.Unet(
    encoder_name='resnet34',  # Choose the encoder backbone, e.g., 'resnet18', 'resnet34', 'resnet50'
    encoder_weights='imagenet',  # Use ImageNet pretraining weights
    in_channels=3,  # Number of input channels (e.g., 3 for RGB images)
    classes=2  # Number of output classes (e.g., 2 for binary segmentation)
)
model1.load_state_dict(checkpoint['model_state_dict'])

In [3]:
transforms = {
    "train": tsfms.Compose([
        tsfms.RandomRotate(60),
        tsfms.RandomSizedCrop(512, frac_range=[0.08, 1]),
        tsfms.RandomHorizontalFlip(),
        #tsfms.RandomIntensityJitter(0.1, 0.1),
        tsfms.Clip(0, 255, 0, 1),
        tsfms.ToTensor(),
    ]),
    "val": tsfms.Compose([
        tsfms.Resize((512, 512)),
        tsfms.Clip(0, 255, 0, 1),
        tsfms.ToTensor()
    ])
}
data_folder = "/cache/fast_data_nas8/swetha"
train_dataset = CarvanaDataset(data_folder, data_limit=10, transforms=transforms)
val_dataset = CarvanaDataset(data_folder, mode='val', data_limit=10, transforms=transforms)
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)  # Create your train data loader
val_loader = DataLoader(val_dataset, batch_size=2, shuffle=True)  # Create your validation data loader

In [9]:
train_dataset[0][0].size()

torch.Size([3, 512, 512])

In [7]:
train_dataset.masks

['/cache/fast_data_nas8/swetha/train_masks/00087a6bd4dc_01_mask.gif',
 '/cache/fast_data_nas8/swetha/train_masks/00087a6bd4dc_02_mask.gif',
 '/cache/fast_data_nas8/swetha/train_masks/00087a6bd4dc_03_mask.gif',
 '/cache/fast_data_nas8/swetha/train_masks/00087a6bd4dc_04_mask.gif',
 '/cache/fast_data_nas8/swetha/train_masks/00087a6bd4dc_05_mask.gif',
 '/cache/fast_data_nas8/swetha/train_masks/00087a6bd4dc_06_mask.gif',
 '/cache/fast_data_nas8/swetha/train_masks/00087a6bd4dc_07_mask.gif',
 '/cache/fast_data_nas8/swetha/train_masks/00087a6bd4dc_08_mask.gif',
 '/cache/fast_data_nas8/swetha/train_masks/00087a6bd4dc_09_mask.gif',
 '/cache/fast_data_nas8/swetha/train_masks/00087a6bd4dc_10_mask.gif']

In [ ]:
i = 4
plt.figure()
plt.imshow(np.transpose(train_dataset[i]['image'].numpy(), (1, 2, 0)))
plt.figure()
plt.imshow(train_dataset[i]['target'].numpy(), cmap='gray')

In [ ]:
img, mask = train_dataset[0]
plt.figure()
plt.imshow(mask, cmap='gray')
img_numpy = img.numpy()
img_numpy = np.transpose(img_numpy, (1, 2, 0))

plt.figure()
plt.imshow(img_numpy)


In [ ]:
for img, mask in val_loader:
    predictions = model1(img)
    plt.figure()
    plt.imshow(np.transpose(img[0].numpy(), (1, 2, 0)))
    predictions_mask = torch.argmax(predictions, dim=1)
    mask_numpy = np.int64(mask)
    predictions_mask = np.int64(predictions_mask)
    plt.figure()
    plt.imshow(predictions_mask[0])
    plt.figure()
    plt.imshow(mask[0])
    mask_numpy = mask.numpy()
    print(mask_numpy.shape)
    print(predictions_mask.shape)
    dice = np.sum(predictions_mask[mask_numpy==1])*2.0 / (np.sum(predictions_mask) + np.sum(mask_numpy))
    print(dice)
    dice = metrics.dice_score(predictions, mask)
    print(dice)
    break

In [ ]:
predictions = model_(img)

In [ ]:
from metrics import dice_score